# Notebook para normalizar os dados de financiamento do BNDES

Objetivo: O objetivo deste notebook é gerar um único arquivo final com os dados de financiamento do BNDES normalizados. Como existem vários arquivos e de fontes diferentes (Portal dados abertos e site BNDES), há grandes divergências entre os nomes das colunas, valores das colunas e tipos das colunas. Dessa forma, os arquivos serão processados para terem as mesmas colunas resultantes. Além disso, também será feito um processamento para agrupar os projetos que foram quebrados em mais de um contrato do mesmo tipo (mesmo CNPJ, número do contrato, descrição do projeto e forma de apoio) para termos o valor real financiado. O arquivo final gerado (bndes_financiamentos.csv) é usado como entrada para o notebook **BNDES - Gerar modelo de escolha de produto** que gera um modelo de predição de qual é o melhor produto BNDES para o cliente.


<img src="images/gerar_modelo_bndes.png">

Neste notebook são lidos 3 arquivos csv: 
- Operações contratadas na forma direta e indireta não automática (2002 a 30.06.2021).csv [Fonte do dado](https://www.bndes.gov.br/wps/portal/site/home/transparencia/centraldedownloads)
- operacoes-financiamento-operacoes-indiretas-automaticas.csv [Fonte do dado](https://dadosabertos.bndes.gov.br/dataset/operacoes-financiamento/resource/9534f677-9525-4bf8-a3aa-fd5d3e152a93)
- operacoes-financiamento-operacoes-nao-automaticas.csv [Fonte do dado](https://dadosabertos.bndes.gov.br/dataset/operacoes-financiamento/resource/332d446e-d340-46ef-af64-ee6f36e7bd50)


A partir deles, iremos manipulá-los para ter apenas um arquivo final com os valores agrupados. O notebook está dividido nas seguintes etapas:
- 1. Lendo os arquivos csv
- 2. Removendo duplicados
- 3. Normalizando os nomes das colunas
- 4. Normalizando os valores das colunas
- 5. Normalizando os tipos das colunas
- 6. Juntando os 3 dataframes em um único
- 7. Agrupando os valores
- 8. Checando os resultados
- 9. Salvando o dataframe
- 10. Lendo o dataframe

### Importando os pacotes necessários

In [1]:
import os
import re
import pandas as pd
from unidecode import unidecode 

### Pegando o path do projeto

In [2]:
dirname = os.getcwd()
path = os.path.join(dirname, "csv")

### Funções de normalização

In [3]:
def normalize_name(name):
    lower_str = name.lower()
    without_symbols_str = re.sub("[-()]", "", lower_str).replace("r$", "reais").strip()
    without_accents_str = unidecode(without_symbols_str)
    without_blank_lst = filter(None, without_accents_str.split(" "))
    return "_".join(without_blank_lst)

def get_rename_dict(names):
    return {name: normalize_name(name) for name in names}

def keep_only_numbers(string):
    return re.sub("[^0-9]", "", string)

def remove_symbols(string):
    return re.sub("[\.\/-]", "", string)

def format_date(date_string):
    date_lst = date_string.split("-")
    date_lst.reverse()
    return "/".join(date_lst)

def merge_dataframes(df1, df2, df3):
    merged_dataframe = pd.concat([df1, df2, df3])
    merged_dataframe.drop_duplicates(inplace=True)
    return merged_dataframe

def get_size_of_company_label(size_of_company):
    if size_of_company == "MICRO":
        label = 0
    elif size_of_company == "PEQUENA":
        label = 1
    elif size_of_company == "MÉDIA":
        label = 2
    elif size_of_company == "GRANDE":
        label = 3
    else:
        label = 4
    return label

def get_max_contract_number(df1, df2):
    max_contract_number_df1 = max(df1.numero_do_contrato)
    max_contract_number_df2 = max(df2.numero_do_contrato)
    max_contract_number = max(max_contract_number_df1, max_contract_number_df2)
    return max_contract_number

def generate_contract_list(df1, df2, df3):
    contract_number = get_max_contract_number(df1, df2)
    return list(range(contract_number + 1, contract_number + df3.shape[0] + 1))

def rename_contract_status(status):
    if status == "LIQUIDADA":
        status = "LIQUIDADO"
    elif status == "ATIVA":
        status = "ATIVO"
    else:
        status = "-"
    return status

def group_contracts(data):
    agg_dict = {column: ("first" if column != "valor_contratado_reais" else "sum") for column in data.columns}
    grouped_contracts_df = data.groupby(["cnpj", "numero_do_contrato", "forma_de_apoio", "produto", "descricao_do_projeto"], as_index=False).agg(agg_dict)
    return grouped_contracts_df

### 1. Lendo os arquivos csv

In [4]:
operacoes_path = os.path.join(path, "Operações contratadas na forma direta e indireta não automática (2002 a 30.06.2021).csv")
operacoes = pd.read_csv(operacoes_path, sep=";", encoding="latin-1", dtype={"CNPJ": object})
operacoes

,Cliente,CNPJ,Descrição do projeto,UF,Município,Município - código,Número do contrato,Data da contratação,Valor contratado R$,Valor desembolsado R$,...,Subsetor CNAE - nome,Setor BNDES,Subsetor BNDES,Porte do cliente,Natureza do cliente,Instituição Financeira Credenciada,CNPJ da instituição financeira credenciada,Tipo de garantia,Tipo de excepcionalidade,Situação do contrato
0,CEREAL COMERCIO EXPORTACAO E REPRESENTACAO AGR...,00012377000160,LIMITE DE CREDITO PARA FINANCIAMENTO A AQUISIC...,GO,RIO VERDE,5218805,20800431,30/06/2021,2.125.000,0,...,"FABRICACAO DE OLEOS VEGETAIS EM BRUTO, EXCETO ...",INDUSTRIA,ALIMENTO E BEBIDA,GRANDE,PRIVADA,----------,----------,REAL,----------,ATIVO
1,S A USINA CORURIPE ACUCAR E ALCOOL,12229415000110,CONTRATACAO DE CREDITO PARA AQUISICAO OU PRODU...,IE,SEM MUNICÍPIO,9999999,20800411,28/06/2021,667.887,0,...,CULTIVO DE CANA-DE-ACUCAR ...,AGROPECUÁRIA,AGROPECUÁRIA,GRANDE,PRIVADA,----------,----------,REAL / PESSOAL,----------,ATIVO
2,S A USINA CORURIPE ACUCAR E ALCOOL,12229415000110,CONTRATACAO DE CREDITO PARA AQUISICAO OU PRODU...,IE,SEM MUNICÍPIO,9999999,20800411,28/06/2021,3.633.680,0,...,CULTIVO DE CANA-DE-ACUCAR ...,AGROPECUÁRIA,AGROPECUÁRIA,GRANDE,PRIVADA,----------,----------,REAL / PESSOAL,----------,ATIVO
3,S A USINA CORURIPE ACUCAR E ALCOOL,12229415000110,CONTRATACAO DE CREDITO PARA AQUISICAO OU PRODU...,IE,SEM MUNICÍPIO,9999999,20800411,28/06/2021,1.267.923,0,...,CULTIVO DE CANA-DE-ACUCAR ...,AGROPECUÁRIA,AGROPECUÁRIA,GRANDE,PRIVADA,----------,----------,REAL / PESSOAL,----------,ATIVO
4,WD AGROINDUSTRIAL LTDA,01105558000102,CONTRATACAO DE LIMITE DE CREDITO PARA FINANCIA...,MG,JOAO PINHEIRO,3136306,20800271,24/06/2021,644.345,0,...,CULTIVO DE MILHO ...,AGROPECUÁRIA,AGROPECUÁRIA,GRANDE,PRIVADA,----------,----------,REAL / PESSOAL,----------,ATIVO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19596,ACEF S/A,46722831000178,AQUISICAO DE EQUIPAMENTOS NACIONAIS E MOBILIAR...,SP,FRANCA,3516200,1255421,09/01/2002,340.201,340.189,...,EDUCACAO SUPERIOR - GRADUACAO ...,COMERCIO/SERVICOS,COMÉRCIO E SERVIÇOS,GRANDE,PRIVADA,BANCO BRADESCO S.A.,60746948000112,DEFINIDA PELO AGENTE FINANCEIRO,----------,LIQUIDADO
19597,INSTITUTO DE DESENVOLVIMENTO SUSTENTAVEL DO BA...,02275306000186,PROJETO DIREITO E CIDADANIA; PROJETO DEMONSTRA...,BA,ITUBERA,2917300,1247221,03/01/2002,706.600,745.030,...,ATIV ATENCAO A SAUDE HUMANA INTEGR C/ASSIST SO...,COMERCIO/SERVICOS,COMÉRCIO E SERVIÇOS,MICRO,PRIVADA,----------,----------,NÃO SE APLICA,----------,LIQUIDADO
19598,INSTITUTO DE DESENVOLVIMENTO SUSTENTAVEL DO BA...,02275306000186,PROJETO DIREITO E CIDADANIA; PROJETO DEMONSTRA...,BA,ITUBERA,2917300,1247221,03/01/2002,603.981,653.835,...,ATIV ATENCAO A SAUDE HUMANA INTEGR C/ASSIST SO...,COMERCIO/SERVICOS,COMÉRCIO E SERVIÇOS,MICRO,PRIVADA,----------,----------,NÃO SE APLICA,----------,LIQUIDADO
19599,INSTITUTO DE DESENVOLVIMENTO SUSTENTAVEL DO BA...,02275306000186,PROJETO DIREITO E CIDADANIA; PROJETO DEMONSTRA...,BA,ITUBERA,2917300,1247221,03/01/2002,75.692,78.500,...,ATIV ATENCAO A SAUDE HUMANA INTEGR C/ASSIST SO...,COMERCIO/SERVICOS,COMÉRCIO E SERVIÇOS,MICRO,PRIVADA,----------,----------,NÃO SE APLICA,----------,LIQUIDADO


In [5]:
nao_automatica_path = os.path.join(path, "operacoes-financiamento-operacoes-nao-automaticas.csv")
nao_automatica = pd.read_csv(nao_automatica_path, sep=";", encoding="utf-8")
nao_automatica

,cliente,cnpj,descricao_do_projeto,uf,municipio,municipio_codigo,numero_do_contrato,data_da_contratacao,valor_contratado_reais,valor_desembolsado_reais,...,subsetor_cnae_nome,setor_bndes,subsetor_bndes,porte_do_cliente,natureza_do_cliente,instituicao_financeira_credenciada,cnpj_da_instituicao_financeira_credenciada,tipo_de_garantia,tipo_de_excepcionalidade,situacao_do_contrato
0,MUNICIPIO DE RIBEIRAO PRETO,56.024.581/0001-56,PROGRAMA DE MODERNIZACAO DA ADMINISTRACAO TRIB...,SP,RIBEIRAO PRETO,3543402,1247021,2002-01-02,"9090000,0","9007445,1",...,ADMINISTRACAO PUBLICA EM GERAL ...,COMERCIO/SERVICOS,COMÉRCIO E SERVIÇOS,GRANDE,ADMINISTRAÇÃO PÚBLICA DIRETA - GOVERNO MUNICIPAL,----------,----------,"OUTRA, DE NATUREZA ESPECÍFICA OU MISTA",----------,LIQUIDADO
1,INSTITUTO DE DESENVOLVIMENTO SUSTENTAVEL DO BA...,02.275.306/0001-86,PROJETO DIREITO E CIDADANIA; PROJETO DEMONSTRA...,BA,ITUBERA,2917300,1247221,2002-01-03,"706600,0","745030,36",...,ATIV ATENCAO A SAUDE HUMANA INTEGR C/ASSIST SO...,COMERCIO/SERVICOS,COMÉRCIO E SERVIÇOS,MICRO,PRIVADA,----------,----------,NÃO SE APLICA,----------,LIQUIDADO
2,INSTITUTO DE DESENVOLVIMENTO SUSTENTAVEL DO BA...,02.275.306/0001-86,PROJETO DIREITO E CIDADANIA; PROJETO DEMONSTRA...,BA,ITUBERA,2917300,1247221,2002-01-03,"75691,59","78500,0",...,ATIV ATENCAO A SAUDE HUMANA INTEGR C/ASSIST SO...,COMERCIO/SERVICOS,COMÉRCIO E SERVIÇOS,MICRO,PRIVADA,----------,----------,NÃO SE APLICA,----------,LIQUIDADO
3,INSTITUTO DE DESENVOLVIMENTO SUSTENTAVEL DO BA...,02.275.306/0001-86,PROJETO DIREITO E CIDADANIA; PROJETO DEMONSTRA...,BA,ITUBERA,2917300,1247221,2002-01-03,"603981,41","653834,69",...,ATIV ATENCAO A SAUDE HUMANA INTEGR C/ASSIST SO...,COMERCIO/SERVICOS,COMÉRCIO E SERVIÇOS,MICRO,PRIVADA,----------,----------,NÃO SE APLICA,----------,LIQUIDADO
4,ACEF S/A,46.722.831/0001-78,AQUISICAO DE EQUIPAMENTOS NACIONAIS E MOBILIAR...,SP,FRANCA,3516200,1255421,2002-01-09,"340201,0","340189,32",...,EDUCACAO SUPERIOR - GRADUACAO ...,COMERCIO/SERVICOS,COMÉRCIO E SERVIÇOS,GRANDE,PRIVADA,BANCO BRADESCO S.A.,60746948000112,DEFINIDA PELO AGENTE FINANCEIRO,----------,LIQUIDADO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19560,HERCULES MOTORES ELETRICOS LTDA,07.442.711/0001-65,"CONTRATACAO DE CREDITO PARA AQUISICAO, COMERCI...",SC,SEM MUNICÍPIO,9999999,19400011,2021-05-27,"736000,0","0,0",...,"FABRICACAO DE MOTORES ELETRICOS, PECAS E ACESS...",INDUSTRIA,MECÂNICA,GRANDE,PRIVADA,----------,----------,PESSOAL,----------,ATIVO
19561,MUELLER FOGOES LTDA.,04.565.361/0001-36,"CONTRATACAO DE CREDITO PARA AQUISICAO, COMERCI...",SC,SEM MUNICÍPIO,9999999,19400011,2021-05-27,"88000,0","0,0",...,FAB FOGAO REFRIGERA MAQ LAVAR SECAR P/USO DOME...,INDUSTRIA,MECÂNICA,GRANDE,PRIVADA,----------,----------,PESSOAL,----------,ATIVO
19562,MUELLER FOGOES LTDA.,04.565.361/0001-36,"CONTRATACAO DE CREDITO PARA AQUISICAO, COMERCI...",SC,SEM MUNICÍPIO,9999999,19400011,2021-05-27,"104564,0","0,0",...,FAB FOGAO REFRIGERA MAQ LAVAR SECAR P/USO DOME...,INDUSTRIA,MECÂNICA,GRANDE,PRIVADA,----------,----------,PESSOAL,----------,ATIVO
19563,MUELLER FOGOES LTDA.,04.565.361/0001-36,"CONTRATACAO DE CREDITO PARA AQUISICAO, COMERCI...",SC,SEM MUNICÍPIO,9999999,19400011,2021-05-28,"1802854,99","0,0",...,FAB FOGAO REFRIGERA MAQ LAVAR SECAR P/USO DOME...,INDUSTRIA,MECÂNICA,GRANDE,PRIVADA,----------,----------,PESSOAL,----------,ATIVO


In [6]:
indireta_automatica_path = os.path.join(path, "operacoes-financiamento-operacoes-indiretas-automaticas.csv")
indireta_automatica = pd.read_csv(indireta_automatica_path, sep=";", encoding="utf-8")
indireta_automatica

,cliente,cpf_cnpj,uf,municipio,municipio_codigo,data_da_contratacao,valor_da_operacao_em_reais,valor_desembolsado_reais,fonte_de_recurso_desembolsos,custo_financeiro,...,subsetor_cnae_agrupado,subsetor_cnae_codigo,subsetor_cnae_nome,setor_bndes,subsetor_bndes,porte_do_cliente,natureza_do_cliente,instituicao_financeira_credenciada,cnpj_do_agente_financeiro,situacao_da_operacao
0,BANCO COOPERATIVO SICOOB S.A.,**.*38.232/0001-**,SP,PEDREGULHO,3537008,2002-01-02,16000,"16000,0",RECURSOS LIVRES - TESOURO,TAXA FIXA,...,Agropecuária,A0119900,CULT PLANTAS LAVOURA TEMPORARIA NAO ESPECIFICA...,AGROPECUÁRIA,AGROPECUÁRIA,MICRO,PRIVADA,BANCO COOPERATIVO SICOOB S.A.,02.038.232.0001-64,LIQUIDADA
1,BANCO REGIONAL DE DESENVOLVIMENTO DO EXTREMO SUL,**.*16.560/0001-**,RS,TRES DE MAIO,4321808,2002-01-02,8047,"8047,0",RECURSOS VINCULADOS - FAT DEPÓSITOS ESPECIAIS,TAXA FIXA,...,Agropecuária,A0100000,"AGRICULTURA, PECUARIA E SERVICOS RELACIONADOS",AGROPECUÁRIA,AGROPECUÁRIA,MICRO,PRIVADA,BANCO REGIONAL DE DESENVOLVIMENTO DO EXTREMO SUL,92.816.560.0001-37,LIQUIDADA
2,BANCO REGIONAL DE DESENVOLVIMENTO DO EXTREMO SUL,**.*16.560/0001-**,RS,ERECHIM,4307005,2002-01-02,8104,"8104,0",RECURSOS VINCULADOS - FAT DEPÓSITOS ESPECIAIS,TAXA FIXA,...,Agropecuária,A0100000,"AGRICULTURA, PECUARIA E SERVICOS RELACIONADOS",AGROPECUÁRIA,AGROPECUÁRIA,MICRO,PRIVADA,BANCO REGIONAL DE DESENVOLVIMENTO DO EXTREMO SUL,92.816.560.0001-37,LIQUIDADA
3,BANCO REGIONAL DE DESENVOLVIMENTO DO EXTREMO SUL,**.*16.560/0001-**,RS,ERECHIM,4307005,2002-01-02,6304,"6304,0",RECURSOS VINCULADOS - FAT DEPÓSITOS ESPECIAIS,TAXA FIXA,...,Agropecuária,A0100000,"AGRICULTURA, PECUARIA E SERVICOS RELACIONADOS",AGROPECUÁRIA,AGROPECUÁRIA,MICRO,PRIVADA,BANCO REGIONAL DE DESENVOLVIMENTO DO EXTREMO SUL,92.816.560.0001-37,LIQUIDADA
4,BANCO REGIONAL DE DESENVOLVIMENTO DO EXTREMO SUL,**.*16.560/0001-**,RS,HUMAITA,4309704,2002-01-02,15000,"15000,0",RECURSOS VINCULADOS - FAT DEPÓSITOS ESPECIAIS,TAXA FIXA,...,Agropecuária,A0100000,"AGRICULTURA, PECUARIA E SERVICOS RELACIONADOS",AGROPECUÁRIA,AGROPECUÁRIA,MICRO,PRIVADA,BANCO REGIONAL DE DESENVOLVIMENTO DO EXTREMO SUL,92.816.560.0001-37,LIQUIDADA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2083358,LUIZ HENRIQUE PEREIRA 09189608941,**.*40.920/0001-**,SC,TREZE DE MAIO,4218400,2021-05-31,4000,"4000,0",RECURSOS LIVRES - PRÓPRIOS,TAXA FIXA,...,Ativ financeira e seguro,K6424704,COOPERATIVAS DE CREDITO RURAL,COMERCIO/SERVICOS,COMÉRCIO E SERVIÇOS,MICRO,PRIVADA,COOPERATIVA CENTRAL DE CREDITO RURAL COM INTER...,07.202.627.0001-74,ATIVA
2083359,LUMA - COMERCIO E PRODUCAO DE ARTEFATOS PLASTI...,**.*94.946/0001-**,PR,SAO JOSE DOS PINHAIS,4125506,2021-05-31,105130,NaN,-,SELIC,...,Borracha e plástico,C2229302,FABRICACAO ARTEFATOS DE MATERIAL PLASTICO PARA...,INDUSTRIA,OUTRAS,MÉDIA,PRIVADA,AGENCIA DE FOMENTO DO PARANA S.A.,03.584.906.0001-99,ATIVA
2083360,LUPATINI & KRAEMER LTDA,**.*74.998/0001-**,RS,CARAZINHO,4304705,2021-05-31,400000,"400000,0",RECURSOS LIVRES - PRÓPRIOS,SELIC,...,Comércio,G4665600,COM ATAC MAQ E EQUIPAMENTOS PARA USO COMERCIAL...,COMERCIO/SERVICOS,COMÉRCIO E SERVIÇOS,PEQUENA,PRIVADA,BANCO BRADESCO S.A.,60.746.948.0001-12,ATIVA
2083361,INDUSTRIAS ROMI S A,**.*20.428/0001-**,SP,RIO DAS PEDRAS,3544004,2021-05-31,763000,"763000,0",RECURSOS LIVRES - FAT,TLP,...,"Veículo, reboque e carroceria",C2949299,FAB OUTRAS PECAS ACESS P/VEIC AUTOMOTORES NAO ...,INDUSTRIA,MATERIAL DE TRANSPORTE,GRANDE,PRIVADA,BANCO SANTANDER (BRASIL) S.A.,90.400.888.0001-42,ATIVA


### 2. Removendo duplicados

In [7]:
operacoes.drop_duplicates(inplace=True)
operacoes.shape

(19315, 34)

In [8]:
nao_automatica.drop_duplicates(inplace=True)
nao_automatica.shape

(19297, 34)

In [9]:
indireta_automatica.drop_duplicates(inplace=True)
indireta_automatica.shape

(1909196, 30)

In [10]:
print(nao_automatica.columns)
print(operacoes.columns)
print(indireta_automatica.columns)

Index(['cliente', 'cnpj', 'descricao_do_projeto', 'uf', 'municipio',
       'municipio_codigo', 'numero_do_contrato', 'data_da_contratacao',
       'valor_contratado_reais', 'valor_desembolsado_reais',
       'fonte_de_recurso_desembolsos', 'custo_financeiro', 'juros',
       'prazo_carencia_meses', 'prazo_amortizacao_meses',
       'modalidade_de_apoio', 'forma_de_apoio', 'produto',
       'instrumento_financeiro', 'inovacao', 'area_operacional', 'setor_cnae',
       'subsetor_cnae_agrupado', 'subsetor_cnae_codigo', 'subsetor_cnae_nome',
       'setor_bndes', 'subsetor_bndes', 'porte_do_cliente',
       'natureza_do_cliente', 'instituicao_financeira_credenciada',
       'cnpj_da_instituicao_financeira_credenciada', 'tipo_de_garantia',
       'tipo_de_excepcionalidade', 'situacao_do_contrato'],
      dtype='object')
Index(['Cliente', 'CNPJ', 'Descrição do projeto', 'UF', 'Município',
       'Município - código', 'Número do contrato', 'Data da contratação',
       'Valor contratado  R$'

### 3. Normalizando os nomes das colunas
Corrigindo os nomes das colunas para ficarem iguais:
- no arquivo de operações diretas e indiretas, são aplicadas normalizações para o nome da coluna ficar: minúsculo, sem acento, sem símbolos e com separação por hífen
- no arquivo de operações indiretas automáticas são criadas novas colunas e adaptadas o nome de outras através de dicionários

In [11]:
operacoes.rename(get_rename_dict(operacoes.columns), axis=1, inplace=True)
operacoes

,cliente,cnpj,descricao_do_projeto,uf,municipio,municipio_codigo,numero_do_contrato,data_da_contratacao,valor_contratado_reais,valor_desembolsado_reais,...,subsetor_cnae_nome,setor_bndes,subsetor_bndes,porte_do_cliente,natureza_do_cliente,instituicao_financeira_credenciada,cnpj_da_instituicao_financeira_credenciada,tipo_de_garantia,tipo_de_excepcionalidade,situacao_do_contrato
0,CEREAL COMERCIO EXPORTACAO E REPRESENTACAO AGR...,00012377000160,LIMITE DE CREDITO PARA FINANCIAMENTO A AQUISIC...,GO,RIO VERDE,5218805,20800431,30/06/2021,2.125.000,0,...,"FABRICACAO DE OLEOS VEGETAIS EM BRUTO, EXCETO ...",INDUSTRIA,ALIMENTO E BEBIDA,GRANDE,PRIVADA,----------,----------,REAL,----------,ATIVO
1,S A USINA CORURIPE ACUCAR E ALCOOL,12229415000110,CONTRATACAO DE CREDITO PARA AQUISICAO OU PRODU...,IE,SEM MUNICÍPIO,9999999,20800411,28/06/2021,667.887,0,...,CULTIVO DE CANA-DE-ACUCAR ...,AGROPECUÁRIA,AGROPECUÁRIA,GRANDE,PRIVADA,----------,----------,REAL / PESSOAL,----------,ATIVO
2,S A USINA CORURIPE ACUCAR E ALCOOL,12229415000110,CONTRATACAO DE CREDITO PARA AQUISICAO OU PRODU...,IE,SEM MUNICÍPIO,9999999,20800411,28/06/2021,3.633.680,0,...,CULTIVO DE CANA-DE-ACUCAR ...,AGROPECUÁRIA,AGROPECUÁRIA,GRANDE,PRIVADA,----------,----------,REAL / PESSOAL,----------,ATIVO
3,S A USINA CORURIPE ACUCAR E ALCOOL,12229415000110,CONTRATACAO DE CREDITO PARA AQUISICAO OU PRODU...,IE,SEM MUNICÍPIO,9999999,20800411,28/06/2021,1.267.923,0,...,CULTIVO DE CANA-DE-ACUCAR ...,AGROPECUÁRIA,AGROPECUÁRIA,GRANDE,PRIVADA,----------,----------,REAL / PESSOAL,----------,ATIVO
4,WD AGROINDUSTRIAL LTDA,01105558000102,CONTRATACAO DE LIMITE DE CREDITO PARA FINANCIA...,MG,JOAO PINHEIRO,3136306,20800271,24/06/2021,644.345,0,...,CULTIVO DE MILHO ...,AGROPECUÁRIA,AGROPECUÁRIA,GRANDE,PRIVADA,----------,----------,REAL / PESSOAL,----------,ATIVO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19596,ACEF S/A,46722831000178,AQUISICAO DE EQUIPAMENTOS NACIONAIS E MOBILIAR...,SP,FRANCA,3516200,1255421,09/01/2002,340.201,340.189,...,EDUCACAO SUPERIOR - GRADUACAO ...,COMERCIO/SERVICOS,COMÉRCIO E SERVIÇOS,GRANDE,PRIVADA,BANCO BRADESCO S.A.,60746948000112,DEFINIDA PELO AGENTE FINANCEIRO,----------,LIQUIDADO
19597,INSTITUTO DE DESENVOLVIMENTO SUSTENTAVEL DO BA...,02275306000186,PROJETO DIREITO E CIDADANIA; PROJETO DEMONSTRA...,BA,ITUBERA,2917300,1247221,03/01/2002,706.600,745.030,...,ATIV ATENCAO A SAUDE HUMANA INTEGR C/ASSIST SO...,COMERCIO/SERVICOS,COMÉRCIO E SERVIÇOS,MICRO,PRIVADA,----------,----------,NÃO SE APLICA,----------,LIQUIDADO
19598,INSTITUTO DE DESENVOLVIMENTO SUSTENTAVEL DO BA...,02275306000186,PROJETO DIREITO E CIDADANIA; PROJETO DEMONSTRA...,BA,ITUBERA,2917300,1247221,03/01/2002,603.981,653.835,...,ATIV ATENCAO A SAUDE HUMANA INTEGR C/ASSIST SO...,COMERCIO/SERVICOS,COMÉRCIO E SERVIÇOS,MICRO,PRIVADA,----------,----------,NÃO SE APLICA,----------,LIQUIDADO
19599,INSTITUTO DE DESENVOLVIMENTO SUSTENTAVEL DO BA...,02275306000186,PROJETO DIREITO E CIDADANIA; PROJETO DEMONSTRA...,BA,ITUBERA,2917300,1247221,03/01/2002,75.692,78.500,...,ATIV ATENCAO A SAUDE HUMANA INTEGR C/ASSIST SO...,COMERCIO/SERVICOS,COMÉRCIO E SERVIÇOS,MICRO,PRIVADA,----------,----------,NÃO SE APLICA,----------,LIQUIDADO


#### Checando se os nomes das colunas os dois arquivos estão agora iguais depois do processamento:

In [12]:
for name1, name2 in zip(operacoes.columns, nao_automatica.columns):
    print(name1, name2)
    assert(name1 == name2)

cliente cliente
cnpj cnpj
descricao_do_projeto descricao_do_projeto
uf uf
municipio municipio
municipio_codigo municipio_codigo
numero_do_contrato numero_do_contrato
data_da_contratacao data_da_contratacao
valor_contratado_reais valor_contratado_reais
valor_desembolsado_reais valor_desembolsado_reais
fonte_de_recurso_desembolsos fonte_de_recurso_desembolsos
custo_financeiro custo_financeiro
juros juros
prazo_carencia_meses prazo_carencia_meses
prazo_amortizacao_meses prazo_amortizacao_meses
modalidade_de_apoio modalidade_de_apoio
forma_de_apoio forma_de_apoio
produto produto
instrumento_financeiro instrumento_financeiro
inovacao inovacao
area_operacional area_operacional
setor_cnae setor_cnae
subsetor_cnae_agrupado subsetor_cnae_agrupado
subsetor_cnae_codigo subsetor_cnae_codigo
subsetor_cnae_nome subsetor_cnae_nome
setor_bndes setor_bndes
subsetor_bndes subsetor_bndes
porte_do_cliente porte_do_cliente
natureza_do_cliente natureza_do_cliente
instituicao_financeira_credenciada instituic

#### Comparando as colunas normalizadas com as colunas do terceiro arquivo (automáticas indiretas)

In [13]:
print("Colunas extras:\n")
for name in indireta_automatica.columns:
    if name not in list(operacoes.columns):
        print(name)

Colunas extras:

cpf_cnpj
valor_da_operacao_em_reais
cnpj_do_agente_financeiro
situacao_da_operacao


In [14]:
print("Colunas faltando:\n")
for name in operacoes.columns:
    if name not in list(indireta_automatica.columns):
        print(name)

Colunas faltando:

cnpj
descricao_do_projeto
numero_do_contrato
valor_contratado_reais
cnpj_da_instituicao_financeira_credenciada
tipo_de_garantia
tipo_de_excepcionalidade
situacao_do_contrato


#### Criando e adaptando as colunas do arquivo de automáticas indireta

Colunas criadas:
- descricao_do_projeto
- numero_do_contrato
- tipo_de_garantia
- tipo_de_excepcionalidade

Colunas alteradas para virarem equivalentes:

- cpf_cnpj == cnpj
- valor_da_operacao_em_reais == valor_contratado_reais
- cnpj_da_instituicao_financeira_credenciada == cnpj_do_agente_financeiro
- situacao_do_contrato == situacao_da_operacao

In [15]:
indireta_automatica["descricao_do_projeto"] = "SEM DESCRICAO"
indireta_automatica["tipo_de_garantia"] = "DEFINIDA PELO AGENTE FINANCEIRO"
indireta_automatica["tipo_de_excepcionalidade"] = "----------"
indireta_automatica["numero_do_contrato"] = generate_contract_list(operacoes, nao_automatica, indireta_automatica)

### 4. Normalizando os valores das colunas

In [16]:
nao_automatica["cnpj"] = nao_automatica["cnpj"].apply(keep_only_numbers)
nao_automatica["data_da_contratacao"] = nao_automatica["data_da_contratacao"].apply(format_date)
nao_automatica["valor_contratado_reais"] = nao_automatica["valor_contratado_reais"].str.replace(",", ".").astype(float).round(1)
nao_automatica["valor_desembolsado_reais"] = nao_automatica["valor_desembolsado_reais"].str.replace(",", ".").astype(float).round(1)
nao_automatica["juros"] = nao_automatica["juros"].str.replace(",", ".").astype(float)

operacoes["valor_contratado_reais"] = operacoes["valor_contratado_reais"].str.replace(".", "").astype(float).round(1)
operacoes["valor_desembolsado_reais"] = operacoes["valor_desembolsado_reais"].str.replace(".", "").astype(float).round(1)
operacoes["juros"] = operacoes["juros"].str.replace(",", ".").astype(float)

indireta_automatica["valor_contratado_reais"] = indireta_automatica["valor_da_operacao_em_reais"].astype(float).round(1)
indireta_automatica["valor_desembolsado_reais"] = indireta_automatica["valor_desembolsado_reais"].str.replace(",", ".").astype(float).round(1)
indireta_automatica["cnpj_da_instituicao_financeira_credenciada"] = indireta_automatica["cnpj_do_agente_financeiro"].apply(keep_only_numbers)
indireta_automatica["situacao_do_contrato"] = indireta_automatica["situacao_da_operacao"].apply(rename_contract_status)
indireta_automatica["cnpj"] = indireta_automatica["cpf_cnpj"].apply(remove_symbols)
indireta_automatica["juros"] = indireta_automatica["juros"].str.replace(",", ".").astype(float)
indireta_automatica.drop(["valor_da_operacao_em_reais", "cnpj_do_agente_financeiro", "situacao_da_operacao", "cpf_cnpj"], axis=1, inplace=True)
indireta_automatica["data_da_contratacao"] = indireta_automatica["data_da_contratacao"].apply(format_date)
indireta_automatica["municipio_codigo"] = indireta_automatica["municipio_codigo"].astype("int64")
indireta_automatica["subsetor_cnae_agrupado"] = indireta_automatica["subsetor_cnae_agrupado"].str.upper()

### 5. Normalizando os tipos das colunas

In [17]:
indireta_automatica.subsetor_cnae_agrupado.value_counts()

TRANSPORTE TERRESTRE               721373
COMÉRCIO                           314445
AGROPECUÁRIA                       293145
CONSTRUÇÃO                         100118
PRODUTOS ALIMENTÍCIOS               74716
ATIV IMOBIL, PROFISSIONAL E ADM     63267
ATIV AUX TRANSPORTE E ENTREGA       34775
MINERAL NÃO METÁLICO                26367
BORRACHA E PLÁSTICO                 24119
PRODUTO DE METAL                    23377
MÁQUINAS E EQUIPAMENTOS             19263
MÓVEIS                              14460
INDÚSTRIA EXTRATIVA                 14294
QUÍMICA                             13536
ÁGUA, ESGOTO E LIXO                 12705
MADEIRA                             12365
ALOJAMENTO E ALIMENTAÇÃO            12253
CONFEC, VESTUÁRIO E ACESSÓRIOS      11674
ATIV FINANCEIRA E SEGURO            10437
METALURGIA                           9435
COQUE, PETRÓLEO E COMBUSTÍVEL        9017
VEÍCULO, REBOQUE E CARROCERIA        8754
BEBIDAS                              7977
SAÚDE E SERV SOCIAL               

In [18]:
operacoes.subsetor_cnae_agrupado.value_counts()

ELETRICIDADE E GÁS                 4765
PRODUTOS ALIMENTÍCIOS              1225
TRANSPORTE AQUAVIÁRIO              1053
COMÉRCIO                            908
COQUE, PETRÓLEO E COMBUSTÍVEL       855
ATIV AUX TRANSPORTE E ENTREGA       832
AGROPECUÁRIA                        736
QUÍMICA                             729
ADMINISTRAÇÃO PÚBLICA               683
METALURGIA                          620
ÁGUA, ESGOTO E LIXO                 588
INFORMAÇÃO E COMUNICAÇÃO            541
VEÍCULO, REBOQUE E CARROCERIA       516
ARTES, CULTURA E ESPORTE            473
ATIV IMOBIL, PROFISSIONAL E ADM     460
CELULOSE E PAPEL                    408
SAÚDE E SERV SOCIAL                 342
MINERAL NÃO METÁLICO                301
TRANSPORTE TERRESTRE                290
CONSTRUÇÃO                          262
FARMOQUÍMICO, FARMACÊUTICO          245
EDUCAÇÃO                            243
MÁQUINAS E EQUIPAMENTOS             229
ATIV FINANCEIRA E SEGURO            204
INDÚSTRIA EXTRATIVA                 186


In [19]:
nao_automatica.subsetor_cnae_agrupado.value_counts()

ELETRICIDADE E GÁS                 4770
PRODUTOS ALIMENTÍCIOS              1225
TRANSPORTE AQUAVIÁRIO              1047
COMÉRCIO                            905
COQUE, PETRÓLEO E COMBUSTÍVEL       855
ATIV AUX TRANSPORTE E ENTREGA       832
QUÍMICA                             730
AGROPECUÁRIA                        726
ADMINISTRAÇÃO PÚBLICA               681
METALURGIA                          620
ÁGUA, ESGOTO E LIXO                 592
INFORMAÇÃO E COMUNICAÇÃO            541
VEÍCULO, REBOQUE E CARROCERIA       516
ARTES, CULTURA E ESPORTE            473
ATIV IMOBIL, PROFISSIONAL E ADM     458
CELULOSE E PAPEL                    410
SAÚDE E SERV SOCIAL                 342
MINERAL NÃO METÁLICO                301
TRANSPORTE TERRESTRE                291
CONSTRUÇÃO                          263
FARMOQUÍMICO, FARMACÊUTICO          244
EDUCAÇÃO                            243
MÁQUINAS E EQUIPAMENTOS             229
ATIV FINANCEIRA E SEGURO            203
INDÚSTRIA EXTRATIVA                 186


In [20]:
operacoes = operacoes.reindex(sorted(operacoes.columns), axis=1)
operacoes.dtypes

area_operacional                               object
cliente                                        object
cnpj                                           object
cnpj_da_instituicao_financeira_credenciada     object
custo_financeiro                               object
data_da_contratacao                            object
descricao_do_projeto                           object
fonte_de_recurso_desembolsos                   object
forma_de_apoio                                 object
inovacao                                       object
instituicao_financeira_credenciada             object
instrumento_financeiro                         object
juros                                         float64
modalidade_de_apoio                            object
municipio                                      object
municipio_codigo                                int64
natureza_do_cliente                            object
numero_do_contrato                              int64
porte_do_cliente            

In [21]:
nao_automatica = nao_automatica.reindex(sorted(nao_automatica.columns), axis=1)
nao_automatica.dtypes

area_operacional                               object
cliente                                        object
cnpj                                           object
cnpj_da_instituicao_financeira_credenciada     object
custo_financeiro                               object
data_da_contratacao                            object
descricao_do_projeto                           object
fonte_de_recurso_desembolsos                   object
forma_de_apoio                                 object
inovacao                                       object
instituicao_financeira_credenciada             object
instrumento_financeiro                         object
juros                                         float64
modalidade_de_apoio                            object
municipio                                      object
municipio_codigo                                int64
natureza_do_cliente                            object
numero_do_contrato                              int64
porte_do_cliente            

In [22]:
indireta_automatica = indireta_automatica.reindex(sorted(indireta_automatica.columns), axis=1)
indireta_automatica.dtypes

area_operacional                               object
cliente                                        object
cnpj                                           object
cnpj_da_instituicao_financeira_credenciada     object
custo_financeiro                               object
data_da_contratacao                            object
descricao_do_projeto                           object
fonte_de_recurso_desembolsos                   object
forma_de_apoio                                 object
inovacao                                       object
instituicao_financeira_credenciada             object
instrumento_financeiro                         object
juros                                         float64
modalidade_de_apoio                            object
municipio                                      object
municipio_codigo                                int64
natureza_do_cliente                            object
numero_do_contrato                              int64
porte_do_cliente            

In [23]:
for ind, (type1, type2, type3) in enumerate(zip(operacoes.dtypes, nao_automatica.dtypes, indireta_automatica.dtypes)):
    print("Index: {}, Dtypes: {}, {}, {}".format(ind, type1, type2, type3))
    assert(type1 == type2 == type3)

Index: 0, Dtypes: object, object, object
Index: 1, Dtypes: object, object, object
Index: 2, Dtypes: object, object, object
Index: 3, Dtypes: object, object, object
Index: 4, Dtypes: object, object, object
Index: 5, Dtypes: object, object, object
Index: 6, Dtypes: object, object, object
Index: 7, Dtypes: object, object, object
Index: 8, Dtypes: object, object, object
Index: 9, Dtypes: object, object, object
Index: 10, Dtypes: object, object, object
Index: 11, Dtypes: object, object, object
Index: 12, Dtypes: float64, float64, float64
Index: 13, Dtypes: object, object, object
Index: 14, Dtypes: object, object, object
Index: 15, Dtypes: int64, int64, int64
Index: 16, Dtypes: object, object, object
Index: 17, Dtypes: int64, int64, int64
Index: 18, Dtypes: object, object, object
Index: 19, Dtypes: int64, int64, int64
Index: 20, Dtypes: int64, int64, int64
Index: 21, Dtypes: object, object, object
Index: 22, Dtypes: object, object, object
Index: 23, Dtypes: object, object, object
Index: 24, 

### 6. Juntando os 3 dataframes em um único

In [24]:
merged_dataframes = merge_dataframes(operacoes, nao_automatica, indireta_automatica)
merged_dataframes

,area_operacional,cliente,cnpj,cnpj_da_instituicao_financeira_credenciada,custo_financeiro,data_da_contratacao,descricao_do_projeto,fonte_de_recurso_desembolsos,forma_de_apoio,inovacao,...,situacao_do_contrato,subsetor_bndes,subsetor_cnae_agrupado,subsetor_cnae_codigo,subsetor_cnae_nome,tipo_de_excepcionalidade,tipo_de_garantia,uf,valor_contratado_reais,valor_desembolsado_reais
0,AREA DE INDUSTRIA E SERVICOS,CEREAL COMERCIO EXPORTACAO E REPRESENTACAO AGR...,00012377000160,----------,TLP,30/06/2021,LIMITE DE CREDITO PARA FINANCIAMENTO A AQUISIC...,-,DIRETA,NÃO,...,ATIVO,ALIMENTO E BEBIDA,PRODUTOS ALIMENTÍCIOS,C1041400,"FABRICACAO DE OLEOS VEGETAIS EM BRUTO, EXCETO ...",----------,REAL,GO,2125000.0,0.0
1,AREA DE INDUSTRIA E SERVICOS,S A USINA CORURIPE ACUCAR E ALCOOL,12229415000110,----------,SELIC,28/06/2021,CONTRATACAO DE CREDITO PARA AQUISICAO OU PRODU...,-,DIRETA,SIM,...,ATIVO,AGROPECUÁRIA,AGROPECUÁRIA,A0113000,CULTIVO DE CANA-DE-ACUCAR ...,----------,REAL / PESSOAL,IE,667887.0,0.0
2,AREA DE INDUSTRIA E SERVICOS,S A USINA CORURIPE ACUCAR E ALCOOL,12229415000110,----------,SELIC,28/06/2021,CONTRATACAO DE CREDITO PARA AQUISICAO OU PRODU...,-,DIRETA,NÃO,...,ATIVO,AGROPECUÁRIA,AGROPECUÁRIA,A0113000,CULTIVO DE CANA-DE-ACUCAR ...,----------,REAL / PESSOAL,IE,3633680.0,0.0
3,AREA DE INDUSTRIA E SERVICOS,S A USINA CORURIPE ACUCAR E ALCOOL,12229415000110,----------,SELIC,28/06/2021,CONTRATACAO DE CREDITO PARA AQUISICAO OU PRODU...,-,DIRETA,NÃO,...,ATIVO,AGROPECUÁRIA,AGROPECUÁRIA,A0113000,CULTIVO DE CANA-DE-ACUCAR ...,----------,REAL / PESSOAL,IE,1267923.0,0.0
4,AREA DE INDUSTRIA E SERVICOS,WD AGROINDUSTRIAL LTDA,01105558000102,----------,SELIC,24/06/2021,CONTRATACAO DE LIMITE DE CREDITO PARA FINANCIA...,RECURSOS LIVRES - PRÓPRIOS,DIRETA,SIM,...,ATIVO,AGROPECUÁRIA,AGROPECUÁRIA,A0111302,CULTIVO DE MILHO ...,----------,REAL / PESSOAL,MG,644345.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2083357,AREA DE OPERACOES E CANAIS DIGITAIS,L DELFINO TOMAZELLI - TRANSPORTES,***129750001**,01401771000153,TAXA FIXA,31/05/2021,SEM DESCRICAO,RECURSOS LIVRES - PRÓPRIOS,INDIRETA,NÃO,...,ATIVO,TRANSPORTE RODOVIÁRIO,TRANSPORTE TERRESTRE,H4930202,TRANSP ROD CARGA EXC PROD PERIG MUDANCA INTER-...,----------,DEFINIDA PELO AGENTE FINANCEIRO,PR,15000.0,15000.0
2083358,AREA DE OPERACOES E CANAIS DIGITAIS,LUIZ HENRIQUE PEREIRA 09189608941,***409200001**,07202627000174,TAXA FIXA,31/05/2021,SEM DESCRICAO,RECURSOS LIVRES - PRÓPRIOS,INDIRETA,NÃO,...,ATIVO,COMÉRCIO E SERVIÇOS,ATIV FINANCEIRA E SEGURO,K6424704,COOPERATIVAS DE CREDITO RURAL,----------,DEFINIDA PELO AGENTE FINANCEIRO,SC,4000.0,4000.0
2083359,AREA DE OPERACOES E CANAIS DIGITAIS,LUMA - COMERCIO E PRODUCAO DE ARTEFATOS PLASTI...,***949460001**,03584906000199,SELIC,31/05/2021,SEM DESCRICAO,-,INDIRETA,NÃO,...,ATIVO,OUTRAS,BORRACHA E PLÁSTICO,C2229302,FABRICACAO ARTEFATOS DE MATERIAL PLASTICO PARA...,----------,DEFINIDA PELO AGENTE FINANCEIRO,PR,105130.0,NaN
2083360,AREA DE OPERACOES E CANAIS DIGITAIS,LUPATINI & KRAEMER LTDA,***749980001**,60746948000112,SELIC,31/05/2021,SEM DESCRICAO,RECURSOS LIVRES - PRÓPRIOS,INDIRETA,NÃO,...,ATIVO,COMÉRCIO E SERVIÇOS,COMÉRCIO,G4665600,COM ATAC MAQ E EQUIPAMENTOS PARA USO COMERCIAL...,----------,DEFINIDA PELO AGENTE FINANCEIRO,RS,400000.0,400000.0


In [25]:
merged_dataframes[["valor_contratado_reais", "data_da_contratacao"]].sort_values(by="valor_contratado_reais")

,valor_contratado_reais,data_da_contratacao
163340,8.000000e+00,04/06/2004
178402,1.000000e+01,21/06/2004
177916,1.000000e+01,21/06/2004
136836,1.800000e+01,09/02/2004
1359798,3.100000e+01,14/12/2012
...,...,...
7622,6.150297e+09,12/12/2012
8376,6.691259e+09,19/07/2012
13099,9.409998e+09,30/07/2009
6498,9.409998e+09,30/07/2009


### 7. Agrupando os valores

In [26]:
grouped_contracts_df = group_contracts(merged_dataframes)
grouped_contracts_df.drop_duplicates(inplace=True)
grouped_contracts_df

,area_operacional,cliente,cnpj,cnpj_da_instituicao_financeira_credenciada,custo_financeiro,data_da_contratacao,descricao_do_projeto,fonte_de_recurso_desembolsos,forma_de_apoio,inovacao,...,situacao_do_contrato,subsetor_bndes,subsetor_cnae_agrupado,subsetor_cnae_codigo,subsetor_cnae_nome,tipo_de_excepcionalidade,tipo_de_garantia,uf,valor_contratado_reais,valor_desembolsado_reais
0,AREA DE OPERACOES E CANAIS DIGITAIS,BANCO DO BRASIL SA,***000000001**,00000000000191,TAXA FIXA,02/01/2002,SEM DESCRICAO,RECURSOS LIVRES - TESOURO,INDIRETA,NÃO,...,LIQUIDADO,AGROPECUÁRIA,AGROPECUÁRIA,A0119900,CULT PLANTAS LAVOURA TEMPORARIA NAO ESPECIFICA...,----------,DEFINIDA PELO AGENTE FINANCEIRO,MS,147393.0,147393.0
1,AREA DE OPERACOES E CANAIS DIGITAIS,BANCO DO BRASIL SA,***000000001**,00000000000191,TAXA FIXA,02/01/2002,SEM DESCRICAO,RECURSOS LIVRES - TESOURO,INDIRETA,NÃO,...,LIQUIDADO,AGROPECUÁRIA,AGROPECUÁRIA,A0119900,CULT PLANTAS LAVOURA TEMPORARIA NAO ESPECIFICA...,----------,DEFINIDA PELO AGENTE FINANCEIRO,MG,60000.0,60000.0
2,AREA DE OPERACOES E CANAIS DIGITAIS,BANCO DO BRASIL SA,***000000001**,00000000000191,TAXA FIXA,02/01/2002,SEM DESCRICAO,RECURSOS LIVRES - TESOURO,INDIRETA,NÃO,...,LIQUIDADO,AGROPECUÁRIA,AGROPECUÁRIA,A0119900,CULT PLANTAS LAVOURA TEMPORARIA NAO ESPECIFICA...,----------,DEFINIDA PELO AGENTE FINANCEIRO,MG,11700.0,11700.0
3,AREA DE OPERACOES E CANAIS DIGITAIS,BANCO DO BRASIL SA,***000000001**,00000000000191,TAXA FIXA,02/01/2002,SEM DESCRICAO,RECURSOS LIVRES - TESOURO,INDIRETA,NÃO,...,LIQUIDADO,AGROPECUÁRIA,AGROPECUÁRIA,A0119900,CULT PLANTAS LAVOURA TEMPORARIA NAO ESPECIFICA...,----------,DEFINIDA PELO AGENTE FINANCEIRO,MS,150000.0,150000.0
4,AREA DE OPERACOES E CANAIS DIGITAIS,BANCO DO BRASIL SA,***000000001**,00000000000191,TAXA FIXA,02/01/2002,SEM DESCRICAO,RECURSOS LIVRES - TESOURO,INDIRETA,NÃO,...,LIQUIDADO,AGROPECUÁRIA,AGROPECUÁRIA,A0119900,CULT PLANTAS LAVOURA TEMPORARIA NAO ESPECIFICA...,----------,DEFINIDA PELO AGENTE FINANCEIRO,MG,70000.0,70000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1918197,AREA DE INDUSTRIA E SERVICOS,DURATEX S.A.,97837181000147,----------,TJLP,13/11/2012,INVESTIMENTOS FIXOS DESTINADOS AO AUMENTO DA C...,RECURSOS LIVRES - FAT / RECURSOS LIVRES - PRÓP...,DIRETA,NÃO,...,ATIVO,OUTRAS,MADEIRA,C1621800,FAB MADEIRA LAMINADA E CHPAS MADEIRA COMPENS P...,----------,PESSOAL,RS,76512000.0,11974295.0
1918198,AREA DE INDUSTRIA E SERVICOS,CAMERA AGROINDUSTRIAL S.A.,98248644000106,----------,TJLP,11/08/2005,REFORCO DE CAPITAL DE GIRO. ...,RECURSOS LIVRES - FAT,DIRETA,NÃO,...,LIQUIDADO,ALIMENTO E BEBIDA,PRODUTOS ALIMENTÍCIOS,C1041400,"FABRICACAO DE OLEOS VEGETAIS EM BRUTO, EXCETO ...",----------,PESSOAL,RS,15000000.0,7523959.0
1918199,AREA DE INDUSTRIA E SERVICOS,CAMERA AGROINDUSTRIAL S.A.,98248644000106,----------,TJLP,18/07/2012,EXPANSAO E MODERNIZACAO DE UNIDADE ESMAGADORA ...,RECURSOS LIVRES - PRÓPRIOS,DIRETA,NÃO,...,ATIVO,ALIMENTO E BEBIDA,PRODUTOS ALIMENTÍCIOS,C1041400,"FABRICACAO DE OLEOS VEGETAIS EM BRUTO, EXCETO ...",----------,REAL / PESSOAL,RS,33749000.0,5177000.0
1918200,AREA DE INDUSTRIA E SERVICOS,COOPERATIVA MISTA SAO LUIZ LTDA.,98338072000148,92816560000137,TAXA FIXA,04/12/2013,AQUISICAO DE PLANTA INDUSTRIAL COM CAPACIDADE ...,RECURSOS LIVRES - TESOURO,INDIRETA,NÃO,...,ATIVO,COMÉRCIO E SERVIÇOS,COMÉRCIO,G4623106,"COMERCIO ATACADISTA DE SEMENTES, FLORES, PLANT...",----------,DEFINIDA PELO AGENTE FINANCEIRO,RS,16803817.0,16803817.0


In [27]:
for column in grouped_contracts_df.columns:
    if grouped_contracts_df[column].dtype == "O":
        grouped_contracts_df[column] = grouped_contracts_df[column].str.strip()

In [28]:
grouped_contracts_df["custo_financeiro"] = grouped_contracts_df["custo_financeiro"].replace({"TJLP", "TLP"})

### 8. Checando os resultados

In [29]:
columns = operacoes.columns
columns_gen = (y for y in columns)
columns_gen_2 = (y for y in columns)
columns_gen_3 = (y for y in columns)

In [30]:
operacoes[next(columns_gen)].value_counts()

AREA DE INDUSTRIA E SERVICOS                           8954
AREA DE ENERGIA                                        5169
AREA DE GESTAO PUBLICA E SOCIOAMBIENTAL                2742
AREA DE SANEAMENTO E TRANSPORTE                        2213
AREA DE OPERACOES E CANAIS DIGITAIS                     129
AREA DE MERC CAP, PARTIC E REEST DE EMPRESAS             50
AREA DE PLANEJAMENTO ESTRATEGICO                         35
AREA DE ESTRUTURACAO DE EMPRESAS E DESINVESTIMENTOS      14
AREA DE ESTRUTURACAO DE PARCERIAS DE INVESTIMENTOS        9
Name: area_operacional, dtype: int64

In [31]:
nao_automatica[next(columns_gen_2)].value_counts()

AREA DE INDUSTRIA E SERVICOS                           8936
AREA DE ENERGIA                                        5158
AREA DE GESTAO PUBLICA E SOCIOAMBIENTAL                2741
AREA DE SANEAMENTO E TRANSPORTE                        2226
AREA DE OPERACOES E CANAIS DIGITAIS                     128
AREA DE MERC CAP, PARTIC E REEST DE EMPRESAS             50
AREA DE PLANEJAMENTO ESTRATEGICO                         35
AREA DE ESTRUTURACAO DE EMPRESAS E DESINVESTIMENTOS      14
AREA DE ESTRUTURACAO DE PARCERIAS DE INVESTIMENTOS        9
Name: area_operacional, dtype: int64

In [32]:
merged_dataframes[next(columns_gen_3)].value_counts()

AREA DE OPERACOES E CANAIS DIGITAIS                    1908955
AREA DE INDUSTRIA E SERVICOS                             13960
AREA DE ENERGIA                                           8632
AREA DE GESTAO PUBLICA E SOCIOAMBIENTAL                   4253
AREA DE SANEAMENTO E TRANSPORTE                           3707
AREA DE MERC CAP, PARTIC E REEST DE EMPRESAS               506
AREA DE PLANEJAMENTO ESTRATEGICO                            64
AREA DE ESTRUTURACAO DE EMPRESAS E DESINVESTIMENTOS         22
AREA DE ESTRUTURACAO DE PARCERIAS DE INVESTIMENTOS          18
Name: area_operacional, dtype: int64

In [33]:
merged_dataframes.columns

Index(['area_operacional', 'cliente', 'cnpj',
       'cnpj_da_instituicao_financeira_credenciada', 'custo_financeiro',
       'data_da_contratacao', 'descricao_do_projeto',
       'fonte_de_recurso_desembolsos', 'forma_de_apoio', 'inovacao',
       'instituicao_financeira_credenciada', 'instrumento_financeiro', 'juros',
       'modalidade_de_apoio', 'municipio', 'municipio_codigo',
       'natureza_do_cliente', 'numero_do_contrato', 'porte_do_cliente',
       'prazo_amortizacao_meses', 'prazo_carencia_meses', 'produto',
       'setor_bndes', 'setor_cnae', 'situacao_do_contrato', 'subsetor_bndes',
       'subsetor_cnae_agrupado', 'subsetor_cnae_codigo', 'subsetor_cnae_nome',
       'tipo_de_excepcionalidade', 'tipo_de_garantia', 'uf',
       'valor_contratado_reais', 'valor_desembolsado_reais'],
      dtype='object')

### 9. Salvando o dataframe

In [34]:
grouped_contracts_df["data_da_contratacao"]

0          02/01/2002
1          02/01/2002
2          02/01/2002
3          02/01/2002
4          02/01/2002
              ...    
1918197    13/11/2012
1918198    11/08/2005
1918199    18/07/2012
1918200    04/12/2013
1918201    11/06/2010
Name: data_da_contratacao, Length: 1918202, dtype: object

In [35]:
grouped_contracts_df.rename({"subsetor_cnae_agrupado": "subsetor_cnae"}, axis=1, inplace=True)
grouped_contracts_df["ano_contratado"] = grouped_contracts_df["data_da_contratacao"].str.split('/').str[2].astype(int)
grouped_contracts_df.drop(["tipo_de_excepcionalidade"], axis=1, inplace=True)

In [36]:
grouped_contracts_df

,area_operacional,cliente,cnpj,cnpj_da_instituicao_financeira_credenciada,custo_financeiro,data_da_contratacao,descricao_do_projeto,fonte_de_recurso_desembolsos,forma_de_apoio,inovacao,...,situacao_do_contrato,subsetor_bndes,subsetor_cnae,subsetor_cnae_codigo,subsetor_cnae_nome,tipo_de_garantia,uf,valor_contratado_reais,valor_desembolsado_reais,ano_contratado
0,AREA DE OPERACOES E CANAIS DIGITAIS,BANCO DO BRASIL SA,***000000001**,00000000000191,TAXA FIXA,02/01/2002,SEM DESCRICAO,RECURSOS LIVRES - TESOURO,INDIRETA,NÃO,...,LIQUIDADO,AGROPECUÁRIA,AGROPECUÁRIA,A0119900,CULT PLANTAS LAVOURA TEMPORARIA NAO ESPECIFICA...,DEFINIDA PELO AGENTE FINANCEIRO,MS,147393.0,147393.0,2002
1,AREA DE OPERACOES E CANAIS DIGITAIS,BANCO DO BRASIL SA,***000000001**,00000000000191,TAXA FIXA,02/01/2002,SEM DESCRICAO,RECURSOS LIVRES - TESOURO,INDIRETA,NÃO,...,LIQUIDADO,AGROPECUÁRIA,AGROPECUÁRIA,A0119900,CULT PLANTAS LAVOURA TEMPORARIA NAO ESPECIFICA...,DEFINIDA PELO AGENTE FINANCEIRO,MG,60000.0,60000.0,2002
2,AREA DE OPERACOES E CANAIS DIGITAIS,BANCO DO BRASIL SA,***000000001**,00000000000191,TAXA FIXA,02/01/2002,SEM DESCRICAO,RECURSOS LIVRES - TESOURO,INDIRETA,NÃO,...,LIQUIDADO,AGROPECUÁRIA,AGROPECUÁRIA,A0119900,CULT PLANTAS LAVOURA TEMPORARIA NAO ESPECIFICA...,DEFINIDA PELO AGENTE FINANCEIRO,MG,11700.0,11700.0,2002
3,AREA DE OPERACOES E CANAIS DIGITAIS,BANCO DO BRASIL SA,***000000001**,00000000000191,TAXA FIXA,02/01/2002,SEM DESCRICAO,RECURSOS LIVRES - TESOURO,INDIRETA,NÃO,...,LIQUIDADO,AGROPECUÁRIA,AGROPECUÁRIA,A0119900,CULT PLANTAS LAVOURA TEMPORARIA NAO ESPECIFICA...,DEFINIDA PELO AGENTE FINANCEIRO,MS,150000.0,150000.0,2002
4,AREA DE OPERACOES E CANAIS DIGITAIS,BANCO DO BRASIL SA,***000000001**,00000000000191,TAXA FIXA,02/01/2002,SEM DESCRICAO,RECURSOS LIVRES - TESOURO,INDIRETA,NÃO,...,LIQUIDADO,AGROPECUÁRIA,AGROPECUÁRIA,A0119900,CULT PLANTAS LAVOURA TEMPORARIA NAO ESPECIFICA...,DEFINIDA PELO AGENTE FINANCEIRO,MG,70000.0,70000.0,2002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1918197,AREA DE INDUSTRIA E SERVICOS,DURATEX S.A.,97837181000147,----------,TJLP,13/11/2012,INVESTIMENTOS FIXOS DESTINADOS AO AUMENTO DA C...,RECURSOS LIVRES - FAT / RECURSOS LIVRES - PRÓP...,DIRETA,NÃO,...,ATIVO,OUTRAS,MADEIRA,C1621800,FAB MADEIRA LAMINADA E CHPAS MADEIRA COMPENS P...,PESSOAL,RS,76512000.0,11974295.0,2012
1918198,AREA DE INDUSTRIA E SERVICOS,CAMERA AGROINDUSTRIAL S.A.,98248644000106,----------,TJLP,11/08/2005,REFORCO DE CAPITAL DE GIRO.,RECURSOS LIVRES - FAT,DIRETA,NÃO,...,LIQUIDADO,ALIMENTO E BEBIDA,PRODUTOS ALIMENTÍCIOS,C1041400,"FABRICACAO DE OLEOS VEGETAIS EM BRUTO, EXCETO ...",PESSOAL,RS,15000000.0,7523959.0,2005
1918199,AREA DE INDUSTRIA E SERVICOS,CAMERA AGROINDUSTRIAL S.A.,98248644000106,----------,TJLP,18/07/2012,EXPANSAO E MODERNIZACAO DE UNIDADE ESMAGADORA ...,RECURSOS LIVRES - PRÓPRIOS,DIRETA,NÃO,...,ATIVO,ALIMENTO E BEBIDA,PRODUTOS ALIMENTÍCIOS,C1041400,"FABRICACAO DE OLEOS VEGETAIS EM BRUTO, EXCETO ...",REAL / PESSOAL,RS,33749000.0,5177000.0,2012
1918200,AREA DE INDUSTRIA E SERVICOS,COOPERATIVA MISTA SAO LUIZ LTDA.,98338072000148,92816560000137,TAXA FIXA,04/12/2013,AQUISICAO DE PLANTA INDUSTRIAL COM CAPACIDADE ...,RECURSOS LIVRES - TESOURO,INDIRETA,NÃO,...,ATIVO,COMÉRCIO E SERVIÇOS,COMÉRCIO,G4623106,"COMERCIO ATACADISTA DE SEMENTES, FLORES, PLANT...",DEFINIDA PELO AGENTE FINANCEIRO,RS,16803817.0,16803817.0,2013


In [37]:
grouped_contracts_df.columns

Index(['area_operacional', 'cliente', 'cnpj',
       'cnpj_da_instituicao_financeira_credenciada', 'custo_financeiro',
       'data_da_contratacao', 'descricao_do_projeto',
       'fonte_de_recurso_desembolsos', 'forma_de_apoio', 'inovacao',
       'instituicao_financeira_credenciada', 'instrumento_financeiro', 'juros',
       'modalidade_de_apoio', 'municipio', 'municipio_codigo',
       'natureza_do_cliente', 'numero_do_contrato', 'porte_do_cliente',
       'prazo_amortizacao_meses', 'prazo_carencia_meses', 'produto',
       'setor_bndes', 'setor_cnae', 'situacao_do_contrato', 'subsetor_bndes',
       'subsetor_cnae', 'subsetor_cnae_codigo', 'subsetor_cnae_nome',
       'tipo_de_garantia', 'uf', 'valor_contratado_reais',
       'valor_desembolsado_reais', 'ano_contratado'],
      dtype='object')

In [38]:
result_dataframe_path = os.path.join(path, "output_csv", "bndes_financiamentos.csv")
grouped_contracts_df.to_csv(result_dataframe_path, index=False, encoding="utf-8", sep=";")

### 10. Lendo o dataframe

In [39]:
pd.read_csv(result_dataframe_path, encoding="utf-8", sep=";")

C:\Users\karina.kato_ifood\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (2,3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,area_operacional,cliente,cnpj,cnpj_da_instituicao_financeira_credenciada,custo_financeiro,data_da_contratacao,descricao_do_projeto,fonte_de_recurso_desembolsos,forma_de_apoio,inovacao,...,situacao_do_contrato,subsetor_bndes,subsetor_cnae,subsetor_cnae_codigo,subsetor_cnae_nome,tipo_de_garantia,uf,valor_contratado_reais,valor_desembolsado_reais,ano_contratado
0,AREA DE OPERACOES E CANAIS DIGITAIS,BANCO DO BRASIL SA,***000000001**,191,TAXA FIXA,02/01/2002,SEM DESCRICAO,RECURSOS LIVRES - TESOURO,INDIRETA,NÃO,...,LIQUIDADO,AGROPECUÁRIA,AGROPECUÁRIA,A0119900,CULT PLANTAS LAVOURA TEMPORARIA NAO ESPECIFICA...,DEFINIDA PELO AGENTE FINANCEIRO,MS,147393.0,147393.0,2002
1,AREA DE OPERACOES E CANAIS DIGITAIS,BANCO DO BRASIL SA,***000000001**,191,TAXA FIXA,02/01/2002,SEM DESCRICAO,RECURSOS LIVRES - TESOURO,INDIRETA,NÃO,...,LIQUIDADO,AGROPECUÁRIA,AGROPECUÁRIA,A0119900,CULT PLANTAS LAVOURA TEMPORARIA NAO ESPECIFICA...,DEFINIDA PELO AGENTE FINANCEIRO,MG,60000.0,60000.0,2002
2,AREA DE OPERACOES E CANAIS DIGITAIS,BANCO DO BRASIL SA,***000000001**,191,TAXA FIXA,02/01/2002,SEM DESCRICAO,RECURSOS LIVRES - TESOURO,INDIRETA,NÃO,...,LIQUIDADO,AGROPECUÁRIA,AGROPECUÁRIA,A0119900,CULT PLANTAS LAVOURA TEMPORARIA NAO ESPECIFICA...,DEFINIDA PELO AGENTE FINANCEIRO,MG,11700.0,11700.0,2002
3,AREA DE OPERACOES E CANAIS DIGITAIS,BANCO DO BRASIL SA,***000000001**,191,TAXA FIXA,02/01/2002,SEM DESCRICAO,RECURSOS LIVRES - TESOURO,INDIRETA,NÃO,...,LIQUIDADO,AGROPECUÁRIA,AGROPECUÁRIA,A0119900,CULT PLANTAS LAVOURA TEMPORARIA NAO ESPECIFICA...,DEFINIDA PELO AGENTE FINANCEIRO,MS,150000.0,150000.0,2002
4,AREA DE OPERACOES E CANAIS DIGITAIS,BANCO DO BRASIL SA,***000000001**,191,TAXA FIXA,02/01/2002,SEM DESCRICAO,RECURSOS LIVRES - TESOURO,INDIRETA,NÃO,...,LIQUIDADO,AGROPECUÁRIA,AGROPECUÁRIA,A0119900,CULT PLANTAS LAVOURA TEMPORARIA NAO ESPECIFICA...,DEFINIDA PELO AGENTE FINANCEIRO,MG,70000.0,70000.0,2002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1918197,AREA DE INDUSTRIA E SERVICOS,DURATEX S.A.,97837181000147,----------,TJLP,13/11/2012,INVESTIMENTOS FIXOS DESTINADOS AO AUMENTO DA C...,RECURSOS LIVRES - FAT / RECURSOS LIVRES - PRÓP...,DIRETA,NÃO,...,ATIVO,OUTRAS,MADEIRA,C1621800,FAB MADEIRA LAMINADA E CHPAS MADEIRA COMPENS P...,PESSOAL,RS,76512000.0,11974295.0,2012
1918198,AREA DE INDUSTRIA E SERVICOS,CAMERA AGROINDUSTRIAL S.A.,98248644000106,----------,TJLP,11/08/2005,REFORCO DE CAPITAL DE GIRO.,RECURSOS LIVRES - FAT,DIRETA,NÃO,...,LIQUIDADO,ALIMENTO E BEBIDA,PRODUTOS ALIMENTÍCIOS,C1041400,"FABRICACAO DE OLEOS VEGETAIS EM BRUTO, EXCETO ...",PESSOAL,RS,15000000.0,7523959.0,2005
1918199,AREA DE INDUSTRIA E SERVICOS,CAMERA AGROINDUSTRIAL S.A.,98248644000106,----------,TJLP,18/07/2012,EXPANSAO E MODERNIZACAO DE UNIDADE ESMAGADORA ...,RECURSOS LIVRES - PRÓPRIOS,DIRETA,NÃO,...,ATIVO,ALIMENTO E BEBIDA,PRODUTOS ALIMENTÍCIOS,C1041400,"FABRICACAO DE OLEOS VEGETAIS EM BRUTO, EXCETO ...",REAL / PESSOAL,RS,33749000.0,5177000.0,2012
1918200,AREA DE INDUSTRIA E SERVICOS,COOPERATIVA MISTA SAO LUIZ LTDA.,98338072000148,92816560000137,TAXA FIXA,04/12/2013,AQUISICAO DE PLANTA INDUSTRIAL COM CAPACIDADE ...,RECURSOS LIVRES - TESOURO,INDIRETA,NÃO,...,ATIVO,COMÉRCIO E SERVIÇOS,COMÉRCIO,G4623106,"COMERCIO ATACADISTA DE SEMENTES, FLORES, PLANT...",DEFINIDA PELO AGENTE FINANCEIRO,RS,16803817.0,16803817.0,2013
